### Data sources
- LEHD Origin-Destination Employment Statistics (LODES): The definition of variable codes, datasets, etc. can be found at the latest [LODES 7.3 Technical Documentation](https://lehd.ces.census.gov/data/lodes/LODES7/LODESTechDoc7.3.pdf). All LEHD Origin-Destination Employment Statistics (LODES) data are available, as described in the LODES documentation above. No changes have been made to the original CSV files. Data are available from 2002 to 2015. See the documentation above for caveats.
- Driving Times and Distances Dataset: Census tracts are 2010 vintage, and the columns are the origin tract, destination travel, travel time in minutes, and travel distance in miles. These data were calculated by the Data Science team at the Urban Institute. See [Github repo](https://github.com/UI-Research/spark-osrm).

In [1]:
import pyspark
import numpy as np
import pandas as pd
from pyspark.sql.functions import *
from pyspark.sql.window import Window
from pyspark.ml.linalg import Vectors
from pyspark.ml.regression import LinearRegression
from pyspark.ml.evaluation import RegressionEvaluator 

from IPython.core.interactiveshell import InteractiveShell
from pyspark.sql.types import StringType

warnings.filterwarnings(action='once')
InteractiveShell.ast_node_interactivity = "all"

In [4]:
sc._conf.getAll() 

[(u'spark.eventLog.enabled', u'true'),
 (u'spark.driver.extraLibraryPath',
  u'/usr/lib/hadoop/lib/native:/usr/lib/hadoop-lzo/lib/native'),
 (u'spark.blacklist.decommissioning.timeout', u'1h'),
 (u'spark.yarn.appMasterEnv.SPARK_PUBLIC_DNS', u'$(hostname -f)'),
 (u'spark.jars.packages', u''),
 (u'spark.executor.cores', u'4'),
 (u'spark.driver.host', u'172.31.39.182'),
 (u'spark.executor.extraJavaOptions',
  u"-verbose:gc -XX:+PrintGCDetails -XX:+PrintGCDateStamps -XX:+UseConcMarkSweepGC -XX:CMSInitiatingOccupancyFraction=70 -XX:MaxHeapFreeRatio=70 -XX:+CMSClassUnloadingEnabled -XX:OnOutOfMemoryError='kill -9 %p'"),
 (u'spark.eventLog.dir', u'hdfs:///var/log/spark/apps'),
 (u'spark.sql.hive.metastore.sharedPrefixes',
  u'com.amazonaws.services.dynamodbv2'),
 (u'spark.sql.warehouse.dir', u'hdfs:///user/spark/warehouse'),
 (u'spark.executor.memory', u'5120M'),
 (u'spark.app.id', u'application_1519960558342_0001'),
 (u'spark.serializer.objectStreamReset', u'100'),
 (u'spark.submit.deployMod

In [5]:
spark = SparkSession.builder \
    .appName('pyspark-exploration') \
    .config('spark.driver.cores', '4') \
    .config('spark.executor.memory', '16gb') \
    .config('spark.executor.cores', '4') \
    .getOrCreate()     

In [2]:
def debug(df):
    """
    Function to pretty print the toDebugString
    """
    for rddstring in df.rdd.toDebugString().split('\n'):
        print rddstring.strip()

### Load and prepare data

Load in data and see what it looks like

In [22]:
drive = spark.read.parquet('s3://lsdm-emr-util/lsdm-data/travel-times/drive_times.parquet')
od = spark.read.parquet('s3://lsdm-emr-util/lsdm-data/lodes/od/od.parquet')
rac = spark.read.parquet('s3://lsdm-emr-util/lsdm-data/lodes/rac/rac.parquet')

In [16]:
print((drive.count(), len(drive.columns)))
drive.take(2)
drive.dtypes

(122004331, 4)


[Row(from_tract=u'36103146402', to_tract=u'42091207003', miles=141.2, minutes=184.1),
 Row(from_tract=u'36103146402', to_tract=u'42091209000', miles=162.7, minutes=203.4)]

[('from_tract', 'string'),
 ('to_tract', 'string'),
 ('miles', 'double'),
 ('minutes', 'double')]

In [23]:
print((od.count(), len(od.columns)))
od.take(2)
od.dtypes

(1577789908, 14)


[Row(w_geocode=u'271630714002025', h_geocode=u'271630712082020', s000=1, sa01=0, sa02=1, sa03=0, se01=0, se02=1, se03=0, si01=1, si02=0, si03=0, createdate=u'20160219', year=2012),
 Row(w_geocode=u'271630714002025', h_geocode=u'271630712083004', s000=1, sa01=0, sa02=1, sa03=0, se01=0, se02=1, se03=0, si01=1, si02=0, si03=0, createdate=u'20160219', year=2012)]

[('w_geocode', 'string'),
 ('h_geocode', 'string'),
 ('s000', 'int'),
 ('sa01', 'int'),
 ('sa02', 'int'),
 ('sa03', 'int'),
 ('se01', 'int'),
 ('se02', 'int'),
 ('se03', 'int'),
 ('si01', 'int'),
 ('si02', 'int'),
 ('si03', 'int'),
 ('createdate', 'string'),
 ('year', 'int')]

In [24]:
print((rac.count(), len(rac.columns)))
rac.take(2)
rac.dtypes

(80870134, 44)


[Row(h_geocode=u'260010001001000', c000=4, ca01=0, ca02=4, ca03=0, ce01=1, ce02=1, ce03=2, cns01=0, cns02=0, cns03=0, cns04=0, cns05=0, cns06=0, cns07=0, cns08=1, cns09=1, cns10=0, cns11=0, cns12=0, cns13=0, cns14=1, cns15=0, cns16=1, cns17=0, cns18=0, cns19=0, cns20=0, cr01=4, cr02=0, cr03=0, cr04=0, cr05=0, cr07=0, ct01=4, ct02=0, cd01=0, cd02=1, cd03=2, cd04=1, cs01=3, cs02=1, createdate=u'20170919', year=2015),
 Row(h_geocode=u'260010001001004', c000=2, ca01=0, ca02=2, ca03=0, ce01=1, ce02=1, ce03=0, cns01=0, cns02=0, cns03=0, cns04=0, cns05=0, cns06=0, cns07=0, cns08=0, cns09=0, cns10=1, cns11=0, cns12=0, cns13=0, cns14=0, cns15=0, cns16=0, cns17=0, cns18=1, cns19=0, cns20=0, cr01=2, cr02=0, cr03=0, cr04=0, cr05=0, cr07=0, ct01=2, ct02=0, cd01=1, cd02=1, cd03=0, cd04=0, cs01=1, cs02=1, createdate=u'20170919', year=2015)]

[('h_geocode', 'string'),
 ('c000', 'int'),
 ('ca01', 'int'),
 ('ca02', 'int'),
 ('ca03', 'int'),
 ('ce01', 'int'),
 ('ce02', 'int'),
 ('ce03', 'int'),
 ('cns01', 'int'),
 ('cns02', 'int'),
 ('cns03', 'int'),
 ('cns04', 'int'),
 ('cns05', 'int'),
 ('cns06', 'int'),
 ('cns07', 'int'),
 ('cns08', 'int'),
 ('cns09', 'int'),
 ('cns10', 'int'),
 ('cns11', 'int'),
 ('cns12', 'int'),
 ('cns13', 'int'),
 ('cns14', 'int'),
 ('cns15', 'int'),
 ('cns16', 'int'),
 ('cns17', 'int'),
 ('cns18', 'int'),
 ('cns19', 'int'),
 ('cns20', 'int'),
 ('cr01', 'int'),
 ('cr02', 'int'),
 ('cr03', 'int'),
 ('cr04', 'int'),
 ('cr05', 'int'),
 ('cr07', 'int'),
 ('ct01', 'int'),
 ('ct02', 'int'),
 ('cd01', 'int'),
 ('cd02', 'int'),
 ('cd03', 'int'),
 ('cd04', 'int'),
 ('cs01', 'int'),
 ('cs02', 'int'),
 ('createdate', 'string'),
 ('year', 'int')]

Make census tract and state columns in origin-destination and residential data

In [25]:
od = od.withColumn('h_tract', substring(od.h_geocode, 0, 11))\
        .withColumn('w_tract', substring(od.w_geocode, 0, 11))\
        .withColumn('h_state', substring(od.h_geocode, 0, 2))\
        .withColumn('w_state', substring(od.w_geocode, 0, 2))

In [43]:
rac = rac.withColumn('h_tract', substring(rac.h_geocode, 0, 11))\
        .withColumn('h_state', substring(rac.h_geocode, 0, 2))

Make state column in drive data

In [27]:
drive = drive.withColumn('from_state', substring(drive.from_tract, 0, 2))\
            .withColumn('to_state', substring(drive.to_tract, 0, 2))

Make "total" and "pct" columns

In [28]:
sa_cols = ['sa01', 'sa02', 'sa03']
se_cols = ['se01', 'se02', 'se03']
si_cols = ['si01', 'si02', 'si03']

In [29]:
od = od.withColumn('sa_total', od.sa01+od.sa02+od.sa03)\
    .withColumn('se_total', od.se01+od.se02+od.se03)\
    .withColumn('si_total', od.si01+od.si02+od.si03)

In [30]:
for cat_ls, cat_name in [(sa_cols, 'sa_total'), (se_cols, 'se_total'), (si_cols, 'si_total')]:
    for col in cat_ls:
        new = col + '_pct'
        od = od.withColumn(new, od[col]/od[cat_name])

Check out lineage and partitions of the two dataframes

In [31]:
debug(od)

(137) MapPartitionsRDD[102] at javaToPython at NativeMethodAccessorImpl.java:0 []
|   MapPartitionsRDD[101] at javaToPython at NativeMethodAccessorImpl.java:0 []
|   MapPartitionsRDD[100] at javaToPython at NativeMethodAccessorImpl.java:0 []
|   FileScanRDD[99] at javaToPython at NativeMethodAccessorImpl.java:0 []


In [32]:
debug(drive)

(17) MapPartitionsRDD[106] at javaToPython at NativeMethodAccessorImpl.java:0 []
|   MapPartitionsRDD[105] at javaToPython at NativeMethodAccessorImpl.java:0 []
|   MapPartitionsRDD[104] at javaToPython at NativeMethodAccessorImpl.java:0 []
|   FileScanRDD[103] at javaToPython at NativeMethodAccessorImpl.java:0 []


In [33]:
od.rdd.getNumPartitions()
drive.rdd.getNumPartitions()

137

17

### Join: origin-destination and driving dataframes  
Assumption: travel time and distance for a census tract is the same for all its comprising block groups.

Repartition od before joining



In [34]:
od = od.repartition(17)

Try working with just California data first, where origin and destination are state code 06  
Join od_ca and drive_ca

In [35]:
od_ca = od.where("h_state == 06" and "w_state == 06")

In [36]:
drive_ca = drive.where('to_state == 06' and 'from_state == 06')

In [37]:
df_ca = od_ca.join(drive_ca, [drive.from_tract == od.h_tract, drive.to_tract == od.w_tract])

In [38]:
df_ca.limit(1).collect()

[Row(w_geocode=u'060014355002011', h_geocode=u'060014001001045', s000=1, sa01=1, sa02=0, sa03=0, se01=0, se02=1, se03=0, si01=0, si02=0, si03=1, createdate=u'20160228', year=2008, h_tract=u'06001400100', w_tract=u'06001435500', h_state=u'06', w_state=u'06', sa_total=1, se_total=1, si_total=1, sa01_pct=1.0, sa02_pct=0.0, sa03_pct=0.0, se01_pct=0.0, se02_pct=1.0, se03_pct=0.0, si01_pct=0.0, si02_pct=0.0, si03_pct=1.0, from_tract=u'06001400100', to_tract=u'06001435500', miles=16.9, minutes=24.2, from_state=u'06', to_state=u'06')]

Now join full od with driving, giving us travel times for each origin-destination pair.  

In [39]:
df = od.join(drive, [drive.from_tract == od.h_tract, drive.to_tract == od.w_tract])

Resulting dataframe is split across 200 partitions, as we can see from the getNumPartitions method

In [40]:
df.rdd.getNumPartitions()

200

In [41]:
debug(df)

(200) MapPartitionsRDD[137] at javaToPython at NativeMethodAccessorImpl.java:0 []
|   MapPartitionsRDD[136] at javaToPython at NativeMethodAccessorImpl.java:0 []
|   MapPartitionsRDD[135] at javaToPython at NativeMethodAccessorImpl.java:0 []
|   ZippedPartitionsRDD2[134] at javaToPython at NativeMethodAccessorImpl.java:0 []
|   MapPartitionsRDD[128] at javaToPython at NativeMethodAccessorImpl.java:0 []
|   ShuffledRowRDD[127] at javaToPython at NativeMethodAccessorImpl.java:0 []
+-(17) MapPartitionsRDD[126] at javaToPython at NativeMethodAccessorImpl.java:0 []
|   ShuffledRowRDD[125] at javaToPython at NativeMethodAccessorImpl.java:0 []
+-(137) MapPartitionsRDD[124] at javaToPython at NativeMethodAccessorImpl.java:0 []
|   MapPartitionsRDD[123] at javaToPython at NativeMethodAccessorImpl.java:0 []
|   FileScanRDD[122] at javaToPython at NativeMethodAccessorImpl.java:0 []
|   MapPartitionsRDD[133] at javaToPython at NativeMethodAccessorImpl.java:0 []
|   ShuffledRowRDD[132] at javaToPyt

### Grouped aggregation: proportion of goods-producing jobs per year

Again, start with California only  

The proportion of jobs in California in Goods Producing industry sectors ("si01") has steadily declined over the past decade as the economy moved away from manufacturing and towards services

In [25]:
agg_ca = df_ca.groupBy("year").agg(sum("si01"), sum('si02'), sum('si03'))
results_ca = agg_ca.collect()

In [26]:
agg_ca_df = pd.DataFrame(results_ca)

In [27]:
agg_ca_df.columns = ['year', 'si01', 'si02', 'si03']
agg_ca_df['total_jobs'] = agg_ca_df.si01 + agg_ca_df.si02 + agg_ca_df.si03

In [28]:
agg_ca_df['pct_si01'] = agg_ca_df.si01 / agg_ca_df.total_jobs

In [29]:
agg_ca_df.sort_values('year')

,year,si01,si02,si03,total_jobs,pct_si01
13,2002,1061707,1033147,3245012,5339866,0.198827
0,2003,1019401,1024673,3263365,5307439,0.192070
6,2004,1028667,1022742,3306224,5357633,0.192000
9,2005,1052749,1053289,3372850,5478888,0.192146
3,2006,1045599,1071275,3441228,5558102,0.188122
1,2007,1025099,1065219,3474935,5565253,0.184196
12,2008,967440,1052308,3553468,5573216,0.173587
8,2009,877083,977576,3534286,5388945,0.162756
10,2010,823689,978019,3610732,5412440,0.152184
11,2011,827560,1009758,3651098,5488416,0.150783


Now do it for the whole country. We see a similar pattern of declining percentage of Goods Producing jobs.

In [30]:
agg_full = df.groupBy("year").agg(sum("si01"), sum('si02'), sum('si03'))
results_full = agg_full.collect()

In [31]:
agg_full_df = pd.DataFrame(results_full)

In [33]:
agg_full_df.columns = ['year', 'si01', 'si02', 'si03']
agg_full_df['total_jobs'] = agg_full_df.si01 + agg_full_df.si02 + agg_full_df.si03

In [34]:
agg_full_df['pct_si01'] = agg_full_df.si01 / agg_full_df.total_jobs

In [35]:
agg_full_df.sort_values('year')

,year,si01,si02,si03,total_jobs,pct_si01
13,2002,9049198,9483453,27533384,46066035,0.196440
0,2003,8865886,9600409,27942515,46408810,0.191039
6,2004,9121645,9926406,29220115,48268166,0.188978
9,2005,9300469,10086742,29813064,49200275,0.189033
3,2006,9447083,10167374,30462147,50076604,0.188653
1,2007,9357236,10167600,30749160,50273996,0.186125
12,2008,9023304,10151678,31044813,50219795,0.179676
8,2009,7960944,9624079,30834938,48419961,0.164415
10,2010,7515555,9468287,31314073,48297915,0.155608
11,2011,7827414,9978429,32872199,50678042,0.154454


### Window function: calculate central moving average number jobs at census tract level (RAC)

Central moving average calculates the average between previous year, current year, and next year

In [56]:
# group by tract level to get larger number of jobs to work with
rac_tract = rac.groupBy("h_tract", "year").agg(sum("c000").alias('c000'))

In [53]:
rac_tract.take(1)

[Row(h_tract=u'48027020701', year=2007, c000=587)]

In [54]:
window = Window.partitionBy("h_tract")\
            .orderBy("year")\
            .rowsBetween(Window.currentRow -1, Window.currentRow + 1)
avg_jobs = avg('c000').over(window)

In [61]:
rac_tract_avg = rac_tract.select('year', 'h_tract', avg_jobs.alias("central_avg_jobs"))\
                    .orderBy('h_tract', 'year')

In [62]:
rac_tract_avg.limit(10).show()

+----+-----------+-----------------+
|year|    h_tract| central_avg_jobs|
+----+-----------+-----------------+
|2002|01001020100|            812.0|
|2003|01001020100|            803.0|
|2004|01001020100|796.3333333333334|
|2005|01001020100|825.3333333333334|
|2006|01001020100|865.6666666666666|
|2007|01001020100|913.3333333333334|
|2008|01001020100|            910.0|
|2009|01001020100|877.3333333333334|
|2010|01001020100|822.6666666666666|
|2011|01001020100|            768.0|
+----+-----------+-----------------+



### Reshape RAC data from long to wide based on year column

Result is total number of jobs for each residence area by year

In [65]:
pivoted_rac_tract = rac_tract.groupby('h_tract')\
    .pivot("year")\
    .sum("c000")

In [67]:
pivoted_rac_tract.limit(5).show()

+-----------+----+----+----+----+----+----+----+----+----+----+----+----+----+----+
|    h_tract|2002|2003|2004|2005|2006|2007|2008|2009|2010|2011|2012|2013|2014|2015|
+-----------+----+----+----+----+----+----+----+----+----+----+----+----+----+----+
|27145010402|1455|1504|1589|1636|1685|1752|1949|1902|1924|2056|2278|2139|2185|2124|
|09007595101|1684|1655|1411|1756|1677|1592|1780|1562|1735|1988|1909|1954|1910|1852|
|06077004204|1341|1415|1412|1476|1087|1496|1389|1341|1359|1043|1109|1205|1373|1304|
|48167723800|1731|1902|1844|1977|1888|1783|1844|1790|1715|2234|2244|2264|1675|1802|
|36055007801|1474|1308|1277|1273|1204|1138|1224|1172|1182|1083|1146| 987|1090|1229|
+-----------+----+----+----+----+----+----+----+----+----+----+----+----+----+----+



### Analysis: How do age and earnings affect distance and time traveled for work in California?

Regression equation 1:  
dist = b0 + b1&ast;sa02_pct + b2&ast;sa03_pct + b3&ast;se02_pct + b4&ast;se03_pct + error  

Regression equation 2:  
minutes = b0 + b1&ast;sa02_pct + b2&ast;sa03_pct + b3&ast;se02_pct + b4&ast;se03_pct + error  

sa02 = % of jobs for workers age 30 to 54  
sa03 = % of jobs for workers age 55 or older  
se02 = % of jobs with earnings 1251/month to 3333/month  
se03 = % of jobs with earnings greater than 3333/month  

Note: 
- First category of each group (age an earnings) are left out of equation to serve as baseline
- Filtering data to 2015 so that each origin/destination pair only counts as one observation

This regression explores the relationship between job composition and travel distance by census block. By regressing travel distance on these four variables, we can evaluate whether age and earnings of jobs in a origin/destination pair has an effect on the travel distance. The results show that there's very little relationship.

R-squared is very low, meaning that very little of the variance in outcome can be explained by regressors.  
RMSE is the standard deviation of the unexplained variance and is in the same units as the outcome variable. It is very high and exceeds the intercept in both regressions.

Start with regression equation 1, for travel distance

In [68]:
df_ca_15 = df_ca.where('year == 2015')

In [69]:
df_ca_reg = df_ca_15[['sa02_pct', 'sa03_pct', 'se02_pct', 'se03_pct', 'miles']]

In [70]:
df_ca_reg_vect = df_ca_reg.rdd.map(lambda x: [Vectors.dense(x[0:4]), x[-1]]).toDF(['features', 'label'])
df_ca_reg_vect.show(5)

+-----------------+-----+
|         features|label|
+-----------------+-----+
|[0.0,0.0,1.0,0.0]| 16.9|
|[1.0,0.0,0.0,1.0]| 45.3|
|[0.0,1.0,0.0,1.0]| 16.3|
|[1.0,0.0,1.0,0.0]| 16.3|
|[0.0,0.0,0.0,0.0]| 16.1|
+-----------------+-----+
only showing top 5 rows



In [71]:
lr = LinearRegression(featuresCol = 'features', labelCol = 'label')

In [72]:
lr_model = lr.fit(df_ca_reg_vect)

In [73]:
# make predictions
predictions = lr_model.transform(df_ca_reg_vect)

In [74]:
# print the coefficients and intercept 
print("Coefficients: %s" % str(lr_model.coefficients))
print("Intercept: %s" % str(lr_model.intercept))

Coefficients: [-3.04770349959,-4.00597846361,1.01359120305,1.67570408979]
Intercept: 27.386367196


In [75]:
# evaluate results
trainingSummary = lr_model.summary
print("RMSE: {}".format(trainingSummary.rootMeanSquaredError))
print("r2: {}".format(trainingSummary.r2))

RMSE: 35.0205139273
r2: 0.00161860037597


Now try regression equation 2, time traveled

In [76]:
df_ca_reg = df_ca_15[['sa02_pct', 'sa03_pct', 'se02_pct', 'se03_pct', 'minutes']]

In [78]:
df_ca_reg_vect = df_ca_reg.rdd.map(lambda x: [Vectors.dense(x[0:4]), x[-1]]).toDF(['features', 'label'])
df_ca_reg_vect.show(5)

+-----------------+-----+
|         features|label|
+-----------------+-----+
|[0.0,0.0,1.0,0.0]| 24.2|
|[1.0,0.0,0.0,1.0]| 60.8|
|[1.0,0.0,1.0,0.0]| 22.2|
|[0.0,1.0,0.0,1.0]| 22.2|
|[0.0,0.0,0.0,0.0]| 24.9|
+-----------------+-----+
only showing top 5 rows



In [79]:
lr = LinearRegression(featuresCol = 'features', labelCol = 'label')

In [80]:
lr_model = lr.fit(df_ca_reg_vect)

In [81]:
# make predictions
predictions = lr_model.transform(df_ca_reg_vect)

In [82]:
# print the coefficients and intercept 
print("Coefficients: %s" % str(lr_model.coefficients))
print("Intercept: %s" % str(lr_model.intercept))

Coefficients: [-3.71211382637,-4.69138754015,1.49404738438,2.66253720749]
Intercept: 37.6802550132


In [83]:
# evaluate results
trainingSummary = lr_model.summary
print("RMSE: {}".format(trainingSummary.rootMeanSquaredError))
print("r2: {}".format(trainingSummary.r2))

RMSE: 43.3768570689
r2: 0.00162553820719
